1. Create Service Principal
   * In Azure Active Directory, go to Properties. Make note of the **Directory ID**.
   * Go to App Registrations and create a New application registration
      * example: airlift-app-registration, Web app/API, https://can-be-literally-anything.com
   * Make note of the **Application ID**.
   * Under Manage >  Certificates & secrets, create and copy a new Client secrets. Make note of the ** Value**.
1. Create Storage Account
   * On the Advanced Tab (1), make sure to enable Hierarchal NameSpace (2).
      <img src="https://www.evernote.com/l/AAFW89nF7OtKb4j798yshtao-a4SVE2vUk4B/image.png" width=300px>
   * Make note of the **Storage Account Name**.
   * Create a Data Lake Gen2 file system on the storage account and make note of the **File System Name**.
   * Under Access control (IAM) add a *Role assignment*, where the role is *Storage Blob Data Contributor (Preview)* assigned to the App Registration previously created.

In [2]:

directoryID = "72f988bf-86f1-41af-91ab-2d7cd011db47"
applicationID = "ecf42e9a-b928-4c7c-86e5-5738df90ea0b"
keyValue = "fill-in"
storageAccountName = "mysynapsestorage"
fileSystemName = "sqlondemand1"

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": applicationID,
           "fs.azure.account.oauth2.client.secret": keyValue,
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(directoryID)}


dbutils.fs.mount(
  source = "abfss://{}@{}.dfs.core.windows.net/".format(fileSystemName, storageAccountName),
  mount_point = "/mnt/{}".format(fileSystemName),
  extra_configs = configs)

Out[2]: True

In [3]:
%fs ls 

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/delta/,delta/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


In [4]:
%fs ls /mnt/sqlondemand1/

path,name,size
dbfs:/mnt/sqlondemand1/Retail Data/,Retail Data/,0
dbfs:/mnt/sqlondemand1/csv/,csv/,0
dbfs:/mnt/sqlondemand1/json/,json/,0
dbfs:/mnt/sqlondemand1/parquet/,parquet/,0


In [5]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/delta/,delta/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


In [6]:
%fs mkdirs /mnt/sqlondemand1/training

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

In [7]:
%fs mkdirs /mnt/sqlondemand1/training 

res7: Boolean = true

In [8]:
%fs cp /mnt/training/ /mnt/sqlondemand1/training/ recurse:true

In [9]:
dbutils.fs.cp("/mnt/training/", "/mnt/sqlondemand1/training/", True)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-635638470334847> in <module> 
 ----> 1 dbutils . fs . cp ( "/mnt/training/" , "/mnt/sqlondemand1/training/" , True ) 

 /local_disk0/tmp/1599313623272-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: java.io.FileNotFoundException: dbfs:/mnt/training/UbiqLog4UCI/32_F/log_█▒█▓-█Á-█▓█░█▒█│.txt
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$2(DatabricksFileSystemV2.scala:773)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:108)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$1(DatabricksFileSystemV2.scala:759)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:430)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:453)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:268)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:453)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:411)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:337)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:453)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.getFileStatus(DatabricksFileSystemV2.scala:759)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:337)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:289)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cpRecursive(DBUtilsCore.scala:185)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cpRecursive$1(DBUtilsCore.scala:179)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cpRecursive$1$adapted(DBUtilsCore.scala:178)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cpRecursive(DBUtilsCore.scala:178)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cpRecursive$1(DBUtilsCore.scala:179)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cpRecursive$1$adapted(DBUtilsCore.scala:178)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cpRecursive(DBUtilsCore.scala:178)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cpRecursive$1(DBUtilsCore.scala:179)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cpRecursive$1$adapted(DBUtilsCore.scala:178)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	